In [ ]:
from dotmap import DotMap
from lark import Lark

import os
import sys
import ccxt
import pandas as pd
import csv
from datetime import datetime
import time
# import pytz
import json
import numpy as np 
from pymongo import MongoClient
from bson.objectid import ObjectId
from pprint import pprint

import logging

logging.basicConfig()

print('python', sys.version)
print('CCXT Version:', ccxt.__version__)

In [ ]:
mongo_client = MongoClient()

In [ ]:
exchanges = DotMap({})
config = False
with open ('config.json') as js: 
        config = json.load(js)

for exkey, exconf in config['exchanges'].items():
        if exconf['enabled'] and not exkey in exchanges:
                excred = exconf['cred']
                for crk, crv in excred.items():
                        if crv[0] == "$" : 
                                excred[crk] = os.getenv(crv[1:])

                exchanges[exkey] = getattr(ccxt, exkey) (excred)

In [185]:
tframe2msec = {
  '1m': 1000 * 60 * 1,
  '5m': 1000 * 60 * 5,
  '15m': 1000 * 60 * 15,
  '30m': 1000 * 60 * 30,
  '1h': 1000 * 60 * 60,
  '4h': 1000 * 60 * 60 * 4,
  '8h': 1000 * 60 * 60 * 8,
  '1d': 1000 * 60 * 60 * 24,
}

def get_now_btimestamp(timeframe, offset=0):
  delta = tframe2msec[timeframe]
  now_stamp = int(datetime.now().timestamp() * 1000)
  now_bstamp = divmod(now_stamp, delta)[0] * delta
  return now_bstamp - delta * offset

btnow = int(datetime(2021,1,1).timestamp() * 1000)
btnow


1609459200000

Loaded klines for kucoinfutures XBTUSDTM 8h
Loaded klines for kucoinfutures XBTUSDTM 8h


In [170]:
# positions = []
env_defaults = DotMap(capital=10000, risk_percent=0.5,nhist=100,exchange_id='kucoinfutures',symbol='XBTUSDTM')
env = env_defaults.copy()
kline_watcher = {'kucoinfutures':{'XBTUSDTM':['8h']}}
klines = DotMap()


Loaded klines for kucoinfutures XBTUSDTM 8h


In [ ]:


from pytz import utc

from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.jobstores.mongodb import MongoDBJobStore
from apscheduler.jobstores.sqlalchemy import SQLAlchemyJobStore
from apscheduler.executors.pool import ThreadPoolExecutor, ProcessPoolExecutor


jobstores = {
    'mongo': MongoDBJobStore(),
    'default': SQLAlchemyJobStore(url='sqlite:///jobs.sqlite')
}
executors = {
    'default': ThreadPoolExecutor(3),
    # 'processpool': ProcessPoolExecutor(5)
}
job_defaults = {
    'coalesce': False,
    'max_instances': 5
}


In [ ]:

logging.getLogger('apscheduler').setLevel(logging.INFO)

apscheduler = BackgroundScheduler(jobstores=jobstores, executors=executors, job_defaults=job_defaults, timezone=utc)

def load_klines():
  for exchange_id, symbols in kline_watcher.items():
    for symbol, timeframes in symbols.items():      
        # print(exchange_id, symbol, timeframes)
        for timeframe in timeframes:
          btnow = get_now_btimestamp(timeframe)
          delta = tframe2msec[timeframe]
          df = klines[exchange_id][symbol][timeframe]
          if (df is None) or (len(df) == 0) or (df.iloc[0].timestamp < (btnow - delta)):
            collection_name = f'{symbol}-{timeframe}'
            db = mongo_client[exchange_id]
            df = pd.DataFrame(
              db[collection_name].find({'timestamp': {'$lt': btnow}}).sort('timestamp',-1).limit(env.nhist)
            )
            df['timestamp'] = pd.to_numeric(df.timestamp)
            # df.set_index(df["timestamp"], inplace=True, drop=True)
            klines[exchange_id][symbol][timeframe] = df
            print(f'Loaded klines for {exchange_id} {symbol} {timeframe}')
          # df.sort_index(axis = 0, inplace=True)

  # for symbol in symbols:
  #   for timeframe in timeframes:
      # method_name = f'{exchange_id}_{symbol}_{timeframe}'
      # print(f'->{method_name}')
      # setattr(kline_dyn_updater, method_name, get_latest_to_db_factory(exchange_id, symbol, timeframe))
      # apscheduler.add_job(f'kline_dyn_updater:{method_name}', 'interval', seconds=10, id=method_name)
      

# if os.path.exists("jobs.sqlite"): os.remove("jobs.sqlite")
# apscheduler.start()

apscheduler.add_job(load_klines, 'interval', seconds=5)
apscheduler.start()

In [ ]:
# apscheduler.start()

logging.getLogger('apscheduler').setLevel(logging.ERROR)

In [ ]:
# list(
#   mongo_client['kucoinfutures']['XBTUSDTM-8h'].find({'timestamp': {'$lt': btnow}}).sort('timestamp', -1)
# )
# df = klines['kucoinfutures']['XBTUSDTM']['8h'] = None
btnow = int(datetime.now().timestamp() * 1000)
pd.to_datetime(btnow, unit='ms', utc=False)
# klines

In [ ]:
grammar = """
start: instruction+

instruction: /!exch(ange)?/ NAME -> set_exchange
          | /!sym(bol)?/ NAME -> set_symbol
          | /!cap(ital)?/ NUMBER -> set_capital
          | /!risk(%|(_percent))/ NUMBER -> set_risk_percent
          | "<" INT NAME  -> get_candle
          | "!"NAME NAME  -> set_env_entry
          | "?env"         -> get_env
          | "?"NAME       -> get_env_entry
          | "reset"        -> reset

NAME:   /\w+/

%import common.ESCAPED_STRING   -> STRING
%import common.SIGNED_NUMBER    -> NUMBER

%import common.LETTER
%import common.INT
%import common.WS
%ignore WS
"""

In [183]:
# exchange = None
# symbol = None

def run_instruction(t):
    global env
    if t.data == 'set_env_entry':
        # print('set_env_entry', *t.children)   # We just pass the color names as-is
        key, val = t.children
        # print('set_env_entry->', key, val)   # We just pass the color names as-is
        env[key.value] = val.value

        print(f'Changing env.{key.value} to {val.value}')
    elif t.data == 'set_exchange':
        env.exchange_id = t.children[1].value
        env.exchange = exchanges[env.exchange_id]
        print(f'Changing exchange to {env.exchange}')
    elif t.data == 'set_symbol':
        env.symbol = t.children[1].value
        print(f'Changing symbol to {env.symbol}')
    elif t.data == 'set_capital':
        env.capital = t.children[1].value
        print(f'Changing capital to {env.capital}')
    elif t.data == 'set_risk_percent':
        env.risk_percent = t.children[1].value
        print(f'Changing risk_percent to {env.risk_percent}')
    elif t.data == 'get_candle':
        offset, timeframe = t.children
        offset, timeframe = offset.value, timeframe.value
        print(f'searching {offset} candle for [{env.exchange_id}][{env.symbol}][{timeframe}]')
        delta = tframe2msec[timeframe]
        timestamp = get_now_btimestamp(timeframe, offset=int(offset))
        df = klines[env.exchange_id][env.symbol][timeframe]
        if df is None:
            print(f'Klines missing for {env.exchange_id} {env.symbol} {timeframe}')
            return None
        if not timestamp in df:
            print(f'Kline missing for {timestamp} {env.exchange_id} {env.symbol} {timeframe}')
        row = df.iloc[timestamp]
        print(f'Retrieved candle {row}')
    elif t.data == 'get_env_entry':
        key = t.children[0].value
        print(env[key])
    elif t.data == 'get_env':      
        # print('get_env', *t.children)   # We just pass the color names as-is
        print(env)
    elif t.data == 'reset':     
        # print('reset', *t.children)   # We just pass the color names as-is
        env = env_defaults.copy()
        print(f'Resetting environment to {env}')
 
    else:
        raise SyntaxError('Unknown instruction: %s' % t.data)


In [184]:
parser = Lark(grammar)
 
def run_trader(program):
    parse_tree = parser.parse(program)
 
    for inst in parse_tree.children:
        run_instruction(inst)
 
# def main():
while True:
    code = input('> ')
    if code in ["q", "quit"]: break
    try:
        run_trader(code)
    except Exception as e:
        print(e)

No terminal matches '-' in the current parser context, at line 1 col 3

< -1 8h
  ^
Expected one of: 
	* INT



In [177]:
# env['x'].value
1643932800000, pd.to_datetime(1643932800000,unit='ms',utc=False)

(1643932800000, Timestamp('2022-02-04 00:00:00'))

In [176]:
df = klines['kucoinfutures']['XBTUSDTM']['8h']
df

,_id,timestamp,date,open,high,low,close,volume
0,61fc6c87dc3f40007fcdfc92,1643904000000,2022-02-03 16:00:00,36888.0,37450.0,36260.0,37309.0,10087802.0
1,61fc6c87dc3f40007fcdfc91,1643875200000,2022-02-03 08:00:00,37010.0,37080.0,36219.0,36888.0,11039712.0
2,61fc6c87dc3f40007fcdfc90,1643846400000,2022-02-03 00:00:00,36875.0,37072.0,36702.0,37011.0,4521137.0
3,61fbfc11dc3f40007fcdc819,1643817600000,2022-02-02 16:00:00,37419.0,37634.0,36573.0,36865.0,12579490.0
4,61fbfc11dc3f40007fcdc818,1643788800000,2022-02-02 08:00:00,38270.0,38723.0,37353.0,37420.0,11081561.0
...,...,...,...,...,...,...,...,...
95,61faff2ac803170a9ec39ed6,1641168000000,2022-01-03 00:00:00,47297.0,47297.0,46726.0,47057.0,3345239.0
96,61faff2ac803170a9ec39ed5,1641139200000,2022-01-02 16:00:00,47325.0,47998.0,46631.0,47300.0,7172236.0
97,61faff2ac803170a9ec39ed4,1641110400000,2022-01-02 08:00:00,47170.0,47504.0,46963.0,47325.0,3670674.0
98,61faff2ac803170a9ec39ed3,1641081600000,2022-01-02 00:00:00,47718.0,47750.0,46664.0,47170.0,4370191.0
